In [ ]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
data_dir = "./Data" # Modify as needed

categories = ['cloudy', 'desert', 'green_area', 'water']  # Modify as needed

images = []
labels = []

for label, category in enumerate(categories):
    category_path = os.path.join(data_dir, category)
    for file_name in os.listdir(category_path):
        file_path = os.path.join(category_path, file_name)
        # Load and preprocess the image
        image = cv2.imread(file_path)
        if image is not None:
            image = cv2.resize(image, (128, 128))  # Resize to a consistent shape
            images.append(image)
            labels.append(label)

# Convert lists to numpy arrays
images = np.array(images) / 255.0  # Normalize pixel values to [0, 1]
labels = np.array(labels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42) # Split data

y_train = to_categorical(y_train, num_classes=len(categories))

y_test = to_categorical(y_test, num_classes=len(categories))

In [ ]:
from tensorflow.keras import layers, models

# Building the model
model = models.Sequential([
    # Convolutional layer 1
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    
    # Convolutional layer 2
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # Convolutional layer 3
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # Flattening the output
    layers.Flatten(),
    
    # Fully connected layer
    layers.Dense(128, activation='relu'),
    
    # Output layer with softmax activation for multi-class classification
    layers.Dense(len(categories), activation='softmax')
])

# Compiling the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Displaying the model summary
model.summary()


In [ ]:
# Creating an ImageDataGenerator object for augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
datagen.fit(X_train)

In [ ]:
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_test, y_test),
                    epochs=10)


In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy}')


In [ ]:
import matplotlib.pyplot as plt

# Plotting training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plotting training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
